In [42]:
%load_ext autoreload
%autoreload 2

from hydroml.data.camels_aus_ds import get_dataset
from hydroml.config.config import Config
from hydroml.models.lstm import HydroLSTM
from hydroml.models import get_model

from hydroml.prediction.prediction import process_and_convert_dataloader_to_xarray
import pandas as pd
from hydroml.evaluation.metrics import get_metrics

config = Config(
                target_features=['streamflow_mmd'],
                dynamic_features=['precipitation_AWAP', 'et_morton_wet_SILO'],
                static_features=[],
                evolving_static_features={'dynamic_feature_mean': {'dynamic_features': ['precipitation_AWAP']}}, 
                evolving_metadata={'observed_target_std': {'target_features': ['streamflow_mmd']}}, # can be set automatically
                batch_size=16,
                cal={'periods' : [['2011-01-01', '2017-01-01']], 'catchment_ids':['410730', '401009']},
                val={'periods' : [['2011-01-01', '2017-01-01']], 'catchment_ids':['410730', '401009']},
                dataset_config={'basepath': 'Z://Data//CAMELS_AUS//preprocessed'}
                )

dataset = get_dataset(config, 'val')
dataloader = dataset.to_dataloader()
model = HydroLSTM(config) # or get_model(config)


# make predictions for all catchments
ds = process_and_convert_dataloader_to_xarray(dataloader, model).squeeze()
metrics = get_metrics(ds)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


c:\Users\sho108\AppData\Local\pypoetry\Cache\virtualenvs\hydroml-dFLAodHf-py3.11\Lib\site-packages\xarray\core\computation.py:818: RuntimeWarning: invalid value encountered in sqrt
  result_data = func(*input_data)
c:\Users\sho108\AppData\Local\pypoetry\Cache\virtualenvs\hydroml-dFLAodHf-py3.11\Lib\site-packages\xarray\core\computation.py:818: RuntimeWarning: invalid value encountered in sqrt
  result_data = func(*input_data)
c:\Users\sho108\AppData\Local\pypoetry\Cache\virtualenvs\hydroml-dFLAodHf-py3.11\Lib\site-packages\xarray\core\computation.py:818: RuntimeWarning: invalid value encountered in sqrt
  result_data = func(*input_data)


In [43]:
metrics.to_dataframe()

,nse,kge,rmse,bias,relative_bias,absolute_bias,nse_sqrt
catchment_id,,,,,,,
401009,-0.154836,-0.886736,0.661786,-0.178361,-1.178361,0.178361,NaN
410730,-0.016786,-0.615367,1.246738,0.192413,-0.807587,0.192413,-0.593795
